In [3]:
import sys
sys.path.append('CLIP-dissect') 

In [4]:
import os
import torch
import torchvision
import torch.nn as nn
import numpy as np
import pandas as pd
import cv2
import glob
import random
import copy
from sklearn.metrics import precision_score, f1_score, recall_score, confusion_matrix

import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import random_split
from torchvision.utils import make_grid

import matplotlib.pyplot as plt
import matplotlib
import similarity
import utils
import data_utils
import clip
from sentence_transformers import SentenceTransformer

%matplotlib inline

In [5]:
dataset_path = "FER2013"
class_names = os.listdir(dataset_path+"/train")
print(class_names)

['disgust', 'happy', 'angry', 'neutral', 'sad', 'surprise', 'fear']


In [6]:
emotions = []
for file_name in glob.glob(dataset_path+'/train/*/*'):
    emotion = file_name.split('/')[-2]
    if emotion not in emotions:
        img = cv2.imread(file_name)
    emotions.append(emotion)
    
emotions = []
for file_name in glob.glob(dataset_path+'/test/*/*'):
    emotion = file_name.split('/')[-2]
    if emotion not in emotions:
        img = cv2.imread(file_name)
    emotions.append(emotion)

In [7]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Resize((48,48))])

train_dataset = ImageFolder(dataset_path+'/train',transform)
train_loader = DataLoader(dataset=train_dataset,batch_size=100*6)
#creating val data loaders
val_dataset = ImageFolder(dataset_path+'/test',transform)
val_loader = DataLoader(dataset=val_dataset,batch_size=100)

In [8]:
from clip_dissect_pipeline import dissect_pipeline

In [9]:
emotion_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [10]:
import models
model_path = 'VGGNet'
checkpoint = torch.load(model_path)
target_model = models.Vgg().to(device)
target_model.load_state_dict(checkpoint["params"])
net = target_model
net

Vgg(
  (conv1a): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2a): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2b): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3a): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3b): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4a): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4b): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1a): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn1b): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2a): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn

In [11]:
def correct_count(output, target, topk=(1,)):
    """Computes the top k corrrect count for the specified values of k"""
    maxk = max(topk)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].contiguous().view(-1).float().sum(0, keepdim=True)
        res.append(correct_k)
    return res

In [12]:
def evaluate(net, dataloader, criterion):
    net = net.eval()
    loss_tr, n_samples = 0.0, 0.0

    y_pred = []
    y_gt = []

    correct_count1 = 0
    correct_count2 = 0

    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # fuse crops and batchsize
        #bs, ncrops, c, h, w = inputs.shape
        #bs, c, h, w = inputs.shape
        #inputs = inputs.view(-1, c, h, w)
        ncrops = 10
        bs = 5
        # forward
        outputs = net(inputs)

        # combine results across the crops
        #outputs = outputs.view(bs, ncrops, -1)
        #outputs = torch.sum(outputs, dim=1) / ncrops

        loss = criterion(outputs, labels)

        # calculate performance metrics
        loss_tr += loss.item()

        # accuracy
        counts = correct_count(outputs, labels, topk=(1, 2))
        correct_count1 += counts[0].item()
        correct_count2 += counts[1].item()

        _, preds = torch.max(outputs.data, 1)
        preds = preds.to("cpu")
        labels = labels.to("cpu")
        n_samples += labels.size(0)

        y_pred.extend(pred.item() for pred in preds)
        y_gt.extend(y.item() for y in labels)

    acc1 = 100 * correct_count1 / n_samples
    acc2 = 100 * correct_count2 / n_samples
    loss = loss_tr / n_samples
    print("--------------------------------------------------------")
    print("Top 1 Accuracy: %2.6f %%" % acc1)
    print("Top 2 Accuracy: %2.6f %%" % acc2)
    print("Loss: %2.6f" % loss)
    print("Precision: %2.6f" % precision_score(y_gt, y_pred, average='micro'))
    print("Recall: %2.6f" % recall_score(y_gt, y_pred, average='micro'))
    print("F1 Score: %2.6f" % f1_score(y_gt, y_pred, average='micro'))
    print("Confusion Matrix:\n", confusion_matrix(y_gt, y_pred), '\n')
    return acc1, acc2

In [11]:
criterion = nn.CrossEntropyLoss()
# Get data with no augmentation
#trainloader, valloader, testloader = get_dataloaders(augment=False)
dataset_path = "FER2013"
mu, st = 0, 1
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((40, 40)),
    transforms.functional.rgb_to_grayscale,
    #transforms.TenCrop(40),
    #transforms.Lambda(lambda tensors: torch.stack([transforms.Normalize(mean=(mu,), std=(st,))(t) for t in tensors])),
    #transforms.Lambda(lambda inputs: inputs.view(-1, inputs.shape[1], inputs.shape[2], inputs.shape[3]))
])
label_corrections = {4: 6, 5:4, 6:5}
target_transform = transforms.Lambda(lambda label: label_corrections[label] if label in label_corrections else label)
val_dataset = ImageFolder(dataset_path+'/test',transform, target_transform)
#plt.imshow(val_dataset[0][0][4].permute(1,2,0))
val_loader = DataLoader(dataset=val_dataset,batch_size=5, shuffle=True)
print("Test")
evaluate(net, val_loader, criterion)

Test


/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


--------------------------------------------------------
Top 1 Accuracy: 69.420451 %
Top 2 Accuracy: 85.775982 %
Loss: 0.281748
Precision: 0.694205
Recall: 0.694205
F1 Score: 0.694205
Confusion Matrix:
 [[ 625    3   72   52  119   12   75]
 [  28   61    2    5   11    0    4]
 [ 121    0  520   36  175   66  106]
 [  26    0   18 1558   47   36   89]
 [ 115    0  103   59  738   17  215]
 [  21    0   81   48   22  635   24]
 [  53    0   38  100  188    8  846]] 



(69.42045137921427, 85.77598216773474)

In [12]:
net1 = copy.deepcopy(net)
weights = net.lin2.weight.data  # Get the weights of the lin1 layer

# Compute the sum of absolute values of weights for each neuron
weight_sums = torch.sum(torch.abs(weights), dim=1)

# Define a threshold to categorize neurons (this is arbitrary and for demonstration)
threshold = weight_sums.mean()  # Use the mean as a simple threshold

# Neurons with weight sum above threshold are "interpretable"
interpretable_indices = torch.where(weight_sums > threshold)[0]
uninterpretable_indices = torch.where(weight_sums <= threshold)[0]

# Modify weights: Increase for interpretable, decrease for uninterpretable
factor = 2  # Example factor to modify weights
for idx in interpretable_indices:
    net1.lin2.weight.data[idx] *= factor

for idx in uninterpretable_indices:
    net1.lin2.weight.data[idx] /= factor
print("modified...")
evaluate(net1, val_loader, criterion)

modified...


/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


--------------------------------------------------------
Top 1 Accuracy: 69.517972 %
Top 2 Accuracy: 85.748119 %
Loss: 0.399206
Precision: 0.695180
Recall: 0.695180
F1 Score: 0.695180
Confusion Matrix:
 [[ 627    3   73   50  124   13   68]
 [  26   67    2    4    9    0    3]
 [ 123    0  523   34  175   69  100]
 [  28    1   19 1557   53   40   76]
 [ 117    0  105   57  754   19  195]
 [  22    1   82   43   19  643   21]
 [  56    1   42  102  202   11  819]] 



(69.51797157982725, 85.7481192532739)

In [13]:
d_probe = 'FER2013'
concept_set = 'CLIP-dissect/data/concept_set.txt' # concept set needs to be path to .txt file
similarity_fn = similarity.soft_wpmi
target_name = 'deep_emotion'
layers = ['conv1a', 'conv1b', 'conv2a', 'conv2b', 'conv3a', 'conv3b', 'conv4a', 'conv4b', 'lin1', 'lin2']
target_layer = 'conv4b'

predicted_emotions, neuron_image_indices, sims = dissect_pipeline(d_probe,concept_set, similarity_fn, target_name, 'lin2')


100%|██████████| 4096/4096 [00:00<00:00, 8856.52it/s]


torch.Size([4096, 25])


In [25]:
res = {}
for tau in np.arange(-1, 2, .25):
    for factor in np.arange(1, 4, .5):  # Example factor to modify weights
        net2 = copy.deepcopy(net)
        for idx in sims.keys():
            # Modify weights: Increase for interpretable, decrease for uninterpretable
            if sims[idx] > tau: # interpretable
                net2.lin2.weight.data[idx] *= factor
            else:               # uninterpretable
                net2.lin2.weight.data[idx] /= factor
        acc1, acc2 = evaluate(net2, val_loader, criterion)
        res[(tau, factor)] = acc1

--------------------------------------------------------
Top 1 Accuracy: 69.420451 %
Top 2 Accuracy: 85.775982 %
Loss: 0.281655
Precision: 0.694205
Recall: 0.694205
F1 Score: 0.694205
Confusion Matrix:
 [[ 625    3   72   52  119   12   75]
 [  28   61    2    5   11    0    4]
 [ 121    0  520   36  175   66  106]
 [  26    0   18 1558   47   36   89]
 [ 115    0  103   59  738   17  215]
 [  21    0   81   48   22  635   24]
 [  53    0   38  100  188    8  846]] 



/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


--------------------------------------------------------
Top 1 Accuracy: 69.573697 %
Top 2 Accuracy: 85.748119 %
Loss: 0.380680
Precision: 0.695737
Recall: 0.695737
F1 Score: 0.695737
Confusion Matrix:
 [[ 621    3   75   54  113   14   78]
 [  26   67    2    4    9    0    3]
 [ 120    0  520   37  166   73  108]
 [  22    1   20 1560   42   40   89]
 [ 115    0  108   60  719   20  225]
 [  17    1   75   44   19  650   25]
 [  54    0   39  104  167   12  857]] 



/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


--------------------------------------------------------
Top 1 Accuracy: 69.545834 %
Top 2 Accuracy: 85.692393 %
Loss: 0.483105
Precision: 0.695458
Recall: 0.695458
F1 Score: 0.695458
Confusion Matrix:
 [[ 622    3   75   54  103   18   83]
 [  26   67    2    4    9    0    3]
 [ 119    0  521   38  161   77  108]
 [  20    1   20 1559   42   43   89]
 [ 116    0  111   63  706   22  229]
 [  16    1   73   42   18  657   24]
 [  51    1   39  104  162   16  860]] 



...

--------------------------------------------------------
Top 1 Accuracy: 68.793536 %
Top 2 Accuracy: 85.371970 %
Loss: 0.335254
Precision: 0.687935
Recall: 0.687935
F1 Score: 0.687935
Confusion Matrix:
 [[ 592    7  111   36  126   26   60]
 [  20   76    3    3    7    0    2]
 [ 102    0  571   25  170   77   79]
 [  23    1   39 1502   53   66   90]
 [ 108    2  155   43  748   24  167]
 [  13    1   86   29   18  665   19]
 [  55    1   67   78  221   27  784]] 



/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


--------------------------------------------------------
Top 1 Accuracy: 68.500975 %
Top 2 Accuracy: 85.246587 %
Loss: 0.371503
Precision: 0.685010
Recall: 0.685010
F1 Score: 0.685010
Confusion Matrix:
 [[ 585    8  115   36  126   29   59]
 [  18   77    4    3    6    1    2]
 [ 100    0  579   20  165   83   77]
 [  24    2   43 1488   54   73   90]
 [ 104    2  161   41  744   31  164]
 [  12    1   85   28   16  670   19]
 [  53    1   73   76  226   30  774]] 



/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


--------------------------------------------------------
Top 1 Accuracy: 68.417386 %
Top 2 Accuracy: 85.093341 %
Loss: 0.411027
Precision: 0.684174
Recall: 0.684174
F1 Score: 0.684174
Confusion Matrix:
 [[ 580    9  121   33  126   32   57]
 [  18   78    4    3    5    1    2]
 [  99    1  585   18  159   86   76]
 [  24    2   45 1478   54   82   89]
 [ 102    2  171   39  743   34  156]
 [  11    1   81   25   15  680   18]
 [  51    1   78   76  225   35  767]] 



In [33]:
69.629 - 69.42

0.20900000000000318

In [29]:
for key in sorted(res, key=res.get, reverse=True):
    print(key, res[key])

(-0.5, 1.5) 69.62942323767066
(0.0, 1.5) 69.62942323767066
(0.25, 1.5) 69.62942323767066
(0.25, 2.5) 69.61549178044024
(0.0, 2.5) 69.58762886597938
(0.5, 1.5) 69.58762886597938
(-1.0, 1.5) 69.57369740874896
(-0.25, 1.5) 69.57369740874896
(0.0, 3.0) 69.57369740874896
(-1.0, 2.0) 69.5458344942881
(-0.75, 1.5) 69.5458344942881
(-0.25, 2.5) 69.5458344942881
(0.75, 1.5) 69.5458344942881
(1.25, 1.5) 69.5458344942881
(-0.25, 3.0) 69.53190303705767
(0.0, 2.0) 69.53190303705767
(0.0, 3.5) 69.53190303705767
(-0.75, 2.0) 69.51797157982725
(-0.75, 2.5) 69.51797157982725
(-0.25, 3.5) 69.51797157982725
(0.5, 2.5) 69.51797157982725
(1.0, 1.5) 69.51797157982725
(-0.5, 2.5) 69.50404012259682
(-1.0, 2.5) 69.49010866536639
(-0.5, 2.0) 69.49010866536639
(-0.75, 3.0) 69.47617720813597
(0.5, 2.0) 69.47617720813597
(-0.25, 2.0) 69.46224575090554
(0.25, 2.0) 69.46224575090554
(-1.0, 1.0) 69.42045137921427
(-1.0, 3.0) 69.42045137921427
(-0.75, 1.0) 69.42045137921427
(-0.5, 1.0) 69.42045137921427
(-0.5, 3.5) 69

In [ ]:
plt.sim

In [ ]:
min_sim = min(sims.values())
max_sim = max(sims.values())
min_sim, max_sim

In [ ]:
res = {}
min_sim = min(sims.values())
max_sim = max(sims.values())

for factor in [2, 3, 4]:  # Example factor to modify weights
    net2 = copy.deepcopy(net)
    for idx in sims.keys():
        # Modify weights: Increase for interpretable, decrease for uninterpretable
        net2.lin2.weight.data[idx] *= factor * (sims[idx] - min_sim) / (max_sim - min_sim)
    acc1, acc2 = evaluate(net2, val_loader, criterion)
    res[(tau, factor)] = acc1

In [ ]:
sorted(res.values())

In [ ]:
res = {}
min_sim = min(sims.values())
max_sim = max(sims.values())
mean_sim = sum(sims.values()) / len(sims)
for factor in np.arange(1, 15):  # Example factor to modify weights
    net2 = copy.deepcopy(net)
    for idx in sims.keys():
        # Modify weights: Increase for interpretable, decrease for uninterpretable
        net2.lin2.weight.data[idx] *= factor * (sims[idx] - min_sim) / (max_sim - min_sim)
    acc1, acc2 = evaluate(net2, val_loader, criterion)
    res[factor] = acc1

In [ ]:
sorted(res.values())

In [21]:
concept_labels = 'CLIP-dissect/data/concept_set_labels.txt'
with open(concept_labels, 'r') as f: 
    s = (f.read()).split('\n')
labels = {}
for item in s:
    items = item.split(':')
    labels[items[0]] = items[1].strip().split(',')
print(labels)

{'Furrowed Brows': ['angry'], 'Raised Brows': ['surprise'], 'Lowered Brows': ['angry'], 'Drooping Brows': ['sad'], 'Relaxed Brows': ['neutral'], 'Wide Open Eyes': ['surprise'], 'Narrowed Eyes': ['angry'], 'Twinkling or Crinkled Eyes': ['happy'], 'Tearful Eyes': ['sad'], 'Closed Eyes': ['sad', 'neutral', 'fear'], 'Relaxed Eyes': ['neutral'], 'Tense Mouth and Jaw': ['angry', 'disgust'], 'Relaxed Mouth and Jaw': ['happy', 'neutral', 'sad'], 'Smile': ['happy'], 'Frown': ['sad'], 'Downturned Mouth': ['sad'], 'Slightly Open Mouth': ['surprise'], 'Wide Open Mouth': ['surprise'], 'Slight Chin Raise': ['surprise', 'neutral'], 'Slightly Dropped Jaw': ['surprise'], 'Flared Nostrils': ['angry', 'disgust'], 'Tightened Facial Muscles': ['angry', 'fear', 'disgust'], 'Compressed Lips': ['angry', 'disgust'], 'Elevated Upper Eyelids': ['surprise'], 'Relaxed Facial Muscles': ['happy', 'neutral']}


In [16]:
from collections import Counter

def get_actual_labels(image_indices, pil_data):
    labels = []
    for idx in image_indices:
        path, _ = pil_data.samples[idx]
        label = path.split('/')[-2]  
        labels.append(label)
    return labels

def show_images(ids):
    num = len(ids)
    fig = plt.figure(figsize=[10, 5])
    axs = fig.subplots(nrows=num//5, ncols=5)
    for i, id in enumerate(ids):
        im, label = pil_data[id]
        im = im.resize([375,375])
        axs[i // 5, i % 5].imshow(im)
        axs[i // 5, i % 5].axis('off')

In [19]:

pil_data = data_utils.get_data(d_probe)
clip_name = 'ViT-B/16'
device = 'cuda'


In [ ]:
net

In [22]:
from collections import defaultdict 
tau = 0
sim_thresh = .15
overall_accuracies = []
interpretables = []
uninterpretables = []
total_accs_interp = []
total_accs_uninterp = []
for layer in layers:
    predicted_emotions, neuron_image_indices, sims = dissect_pipeline(d_probe,concept_set, similarity_fn, target_name, layer)
    accuracies = defaultdict(dict)
    for neuron, predicted_emotion in zip(neuron_image_indices.keys(), predicted_emotions):
        neuron_id = neuron
        image_indices = neuron_image_indices[neuron]
        actual_labels = get_actual_labels(image_indices, pil_data)
        sim = sims[int(neuron_id.cpu().numpy())]
        accuracies[int(neuron_id)]['sim'] = sim
        correct_count = 0
        for label in actual_labels:
            if label in labels[predicted_emotion]:
                correct_count += 1
        # Calculate the accuracy for this neuron
        accuracy = correct_count / len(actual_labels)
        accuracies[int(neuron_id)]['accuracy'] = accuracy

    # Calculate overall accuracy
    overall_accuracy = sum(d['accuracy'] for d in accuracies.values()) / len(accuracies)
    print(overall_accuracy, len(accuracies))
    overall_accuracies.append(overall_accuracy)
    # Calculate interpretable accuracy
    interpretable = 0
    uninterpretable = 0
    acc_interpretable = 0
    acc_uninterpretable = 0
    for d in accuracies.values():
        if d['sim'] > tau:
            interpretable += 1
            acc_interpretable += d['accuracy']
        else:
            uninterpretable += 1
            acc_uninterpretable += d['accuracy']
    interpretables.append(interpretable)
    uninterpretables.append(uninterpretable)
    print(layer)
    total_acc_interp = acc_interpretable / interpretable if interpretable else 0
    total_acc_uninterp = acc_uninterpretable / uninterpretable if uninterpretable else 0
    total_accs_interp.append(total_acc_interp)
    total_accs_uninterp.append(total_acc_uninterp)
    

100%|██████████| 64/64 [00:00<00:00, 3559.44it/s]


torch.Size([64, 25])
0.21562499999999984 64
conv1a


100%|██████████| 64/64 [00:00<00:00, 4878.43it/s]


torch.Size([64, 25])
0.2999999999999999 64
conv1b


100%|██████████| 128/128 [00:00<00:00, 4108.19it/s]


...

torch.Size([512, 25])
0.2833984374999996 512
conv4b


100%|██████████| 4096/4096 [00:00<00:00, 7766.94it/s]


torch.Size([4096, 25])
0.3585693359374982 4096
lin1


100%|██████████| 4096/4096 [00:00<00:00, 7871.50it/s]


torch.Size([4096, 25])
0.4342285156249987 4096
lin2


In [34]:
print("f\n\nj")

f

j


In [46]:
df = pd.DataFrame({'num_interp':interpretables, 'num_uninterp':uninterpretables, \
                  'overall_acc':overall_accuracies, 'interp_acc':total_accs_interp, \
                  'uninterp_acc':total_accs_uninterp}, index=layers)
df['prop_interp'] = df['num_interp'] / (df['num_interp'] + df['num_uninterp'])
pd.options.display.float_format = "{:,.2f}".format
df

,num_interp,num_uninterp,overall_acc,interp_acc,uninterp_acc,prop_interp
conv1a,64,0,0.22,0.22,0.00,1.00
conv1b,64,0,0.30,0.30,0.00,1.00
conv2a,127,1,0.21,0.21,0.30,0.99
conv2b,127,1,0.28,0.27,0.90,0.99
conv3a,254,2,0.26,0.26,0.05,0.99
conv3b,255,1,0.23,0.23,0.30,1.00
conv4a,492,20,0.28,0.29,0.21,0.96
conv4b,494,18,0.28,0.28,0.32,0.96
lin1,3489,607,0.36,0.38,0.25,0.85
lin2,3835,261,0.43,0.45,0.24,0.94


In [ ]:
tau = 1
sim_thresh = .15
detailed_results = {}

for neuron, predicted_emotion in zip(neuron_image_indices.keys(), predicted_emotions):
    neuron_id = neuron
    image_indices = neuron_image_indices[neuron]
    actual_labels = get_actual_labels(image_indices, pil_data)
    sim = sims[int(neuron_id.cpu().numpy())]

    detailed_results[int(neuron_id)] = {
        'predicted_emotion': predicted_emotion,
        'actual_labels': actual_labels,
        'top_image_indices': image_indices,
        'sim': sim
    }
    #label_embed = embedding_model.encode(actual_labels)
    #concept_embed = embedding_model.encode([predicted_emotion] * len(actual_labels))
    #mp_cos = np.mean(np.sum(label_embed*concept_embed, axis=1) > sim_thresh)
    correct_count = 0
    for label in actual_labels:
        if label in labels[predicted_emotion]:
            correct_count += 1
    # Calculate the accuracy for this neuron
    accuracy = correct_count / len(actual_labels)
    detailed_results[int(neuron_id)]['accuracy'] = accuracy

# Calculate overall accuracy
overall_accuracy = sum(d['accuracy'] for d in detailed_results.values()) / len(detailed_results)
# Calculate interpretable accuracy
interpretable = 0
uninterpretable = 0
acc_interpretable = 0
acc_uninterpretable = 0
for d in detailed_results.values():
    if d['sim'] > tau:
        interpretable += 1
        acc_interpretable += d['accuracy']
    else:
        uninterpretable += 1
        acc_uninterpretable += d['accuracy']
print(target_layer)
print(f'num interpretable: {interpretable}')
print(f'num uninterpretable: {uninterpretable}')
total_acc_interp = acc_interpretable / interpretable if interpretable else 0
total_acc_uninterp = acc_uninterpretable / uninterpretable if uninterpretable else 0
# Print the detailed results
i = 0
for neuron_id, results in detailed_results.items():
    i += 1
    if i >= 0:
        break
    print(f"Neuron {neuron_id}: Predicted Emotion = {results['predicted_emotion']}, Accuracy = {results['accuracy']}")
    for image_idx, actual_label in zip(results['top_image_indices'], results['actual_labels']):
        print(f"    Image Index {image_idx}: Actual Label = {actual_label}")

print(f"Overall accuracy: {overall_accuracy}")
print(f"Overall interpretable accuracy: {total_acc_interp}")
print(f"Overall uninterpretable accuracy: {total_acc_uninterp}")

    


In [ ]:
ids = detailed_results[2275]['top_image_indices']
show_images(ids)